# Granular Aggregations

We have now covered table level aggregations, but this is rarely of use for our reporting or analysis needs.

Therefore, although helpful, in most circumstances we are likely to need to levels of aggregation which are inbetween table level and raw data.

For example, maybe we wish to see a year on year breakdown of sales totals, or the volume of sales per employee.  In each of these circumstances we need to add a layer into our aggregation.

In order to do this we utilise the <code>GROUP BY</code> keywords.

To best remember this more easily we can refer to the fact that SQL is declarative and think about "grouping sales by year" and "grouping sales by employee".

For these demo scripts we will use a slightly different table, one that includes sales information.

We can have a quick look at the table using TOP:

In [ ]:
select top 10 *
from JupyterDatabase.sales.salesOrderHeader;

We can see that there are employees making sales (salesPersonID), we have the dates the orders were placed, and the value of the order.

Effectively, we are interested in these columns:

In [ ]:
select top 10 salesPersonID, orderDate, totalDue
from JupyterDatabase.sales.salesOrderHeader;

At this point we shall also perform a quick row count on the table so that we know what we're dealing with:

In [ ]:
select count(*)
from JupyterDatabase.sales.salesOrderHeader;

As you can see, this isn't the largest table (I've seen them in the billions of rows, this itself is just a subset of a table from another database) but it's also by no means small and, at around 1.3 million rows, not something we could comfortably place into Excel as a whole either. Therefore this makes it a good candidate for doing some more granular aggregations in SQL Server itself.

The query we will attempt to create in this section is one that meets the following requirements:

-   Total Sales
    -   By both Month of Sale and SalesPersonID
    -   Total Sales per SalesPersonID per month must be greater than £150,000
    -   For the Year 2018
    -   Ordered first by Month and then SalesPersonID

#### Group By

The first step towards our query goal is to introduce a new set of keywords; GROUP BY.

These, as the keywords imply, allow us to group our data into granular levels.  Therefore if we wish to group our sales by SalesPersonID then we need to obtain the Total Sales (sum of the totalDue) and then group the output by SalesPersonID:

In [ ]:
select sum(totalDue) as totalSales
from JupyterDatabase.sales.salesOrderHeader
group by salesPersonID;

This works perfectly with just one exception, we can't see to which salesPersonID each total belongs.  To achieve this we can add the salesPersonID column to the SELECT line:

In [ ]:
select salesPersonID, sum(totalDue) as totalSales
from JupyterDatabase.sales.salesOrderHeader
group by salesPersonID;

Using the same method we could also display totalSales by Year, remembering how we obtain the year portion of the orderDate using either <code>DATEPART(yy, orderDate)</code> or <code>YEAR(orderDate)</code>:

In [ ]:
select year(orderDate) as orderYear, sum(totalDue) as totalSales
from JupyterDatabase.sales.salesOrderHeader
group by year(orderDate);

As a critical note to the above I will make mention back to the Processing Order of a query in SQL Server.  We need to remember that only an ORDER BY can make use of a column alias and therefore we need to write the full <code>year(orderDate)</code> in the GROUP BY clause.

Grouping does not have to be restricted to just one level and therefore we can add more levels into our query as we wish.  The only thing we need to remember is that all non-aggregated columns in the SELECT clause must also be present in the GROUP BY clause, otherwise you will receive an error informing you that this has happened:

In [ ]:
select salesPersonID, sum(totalDue) as totalSales -- Column is selected (salesPersonID) and Aggregate is used (SUM(totalDue))
from JupyterDatabase.sales.salesOrderHeader; -- Note, no GROUP BY clause will result in error

`Msg 8120, Level 16, State 1, Line 19`

`Column 'JupyterDatabase.sales.salesOrderHeader.salesPersonID' is invalid in the select list because it is not contained in either an aggregate function or the GROUP BY clause.`

You can also see the error message provides the full column designation of 'JupyterDatabase.sales.salesOrderHeader.salesPersonID', further emphasising why we prefer to alias where possible to avoid our needing to use similar long hand syntax.

Anyway, let's correct our query but also add another layer, therefore fulfilling part of our query criteria:

-   **Total Sales**
    -   **By both Month and SalesPersonID**
    -   Total Sales per SalesPersonID per month must be greater than £150,000
    -   For the Year 2018
    -   Ordered first by Month and then SalesPersonID

In [ ]:
select month(orderDate) as orderMonth, salesPersonID, sum(totalDue) as totalSales
from JupyterDatabase.sales.salesOrderHeader
group by month(orderDate), salesPersonID;

This is syntactically correct yet looks wrong as the numbers seem too high? This is because we have not filtered our dataset as requested:

-   Total Sales
    -   By both Month and SalesPersonID
    -   Total Sales per SalesPersonID per month must be greater than £150,000
    -   **For the Year 2018**
    -   Ordered first by Month and then SalesPersonID

Luckily for us, the syntax for WHERE, and for that matter ORDER BY, do not change. The key is simply knowing at which point in the query we insert them.

ORDER BY is always at the end and WHERE is always after the tables (whether this be a FROM or a JOIN, which will come later).

So we can slot our WHERE clause in as follows:

In [ ]:
select month(orderDate) as orderMonth, salesPersonID, sum(totalDue) as totalSales
from JupyterDatabase.sales.salesOrderHeader
where orderDate between '2018-01-01' and '2018-12-31'
group by month(orderDate), salesPersonID;

Note - as we only have a date and not a time portion within our OrderDate column we can safely use 2018-12-31 within our WHERE clause.

So, whilst we're here we shall add our ORDER BY to the bottom of the query (ORDER BY always sits at the end) in order to satisfy the following:

-   Total Sales
    -   By both Month and SalesPersonID
    -   Total Sales per SalesPersonID per month must be greater than £150,000
    -   For the Year 2018
    -   **Ordered first by Month and then SalesPersonID**

Not forgetting that we ARE allowed to use column aliases within our ORDER BY clause:

In [ ]:
select month(orderDate) as orderMonth, salesPersonID, sum(totalDue) as totalSales
from JupyterDatabase.sales.salesOrderHeader
where orderDate between '2018-01-01' and '2018-12-31'
group by month(orderDate), salesPersonID
order by orderMonth, salesPersonID;

This leaves us with just a single criteria remaining:

-   Total Sales
    -   By both Month and SalesPersonID
    -   **Total Sales per SalesPersonID per month must be greater than £150,000**
    -   For the Year 2018
    -   Ordered first by Month and then SalesPersonID

So how can we do this?  Logically we would think that it's simply a filter?  Let's try:

In [ ]:
select month(orderDate) as orderMonth, salesPersonID, sum(totalDue) as totalSales
from JupyterDatabase.sales.salesOrderHeader
where orderDate between '2018-01-01' and '2018-12-31'
and sum(totalSales) > 150000
group by month(orderDate), salesPersonID
order by orderMonth, salesPersonID;

This does NOT work and will yield an error. Why is this?

Well, remember back to the Processing Order again and we can see our answer:

3.  WHERE
4.  GROUP BY

Therefore we have processed the WHERE clause before we process the GROUP BY. This means that we cannot then filter on the GROUP BY using WHERE.

Hence we need another keyword to help us out.

#### HAVING

This is a special keyword reserved for GROUP BY queries and cannot be used with anything else.

It is written after the GROUP BY and, crucially, also processed after the GROUP BY in our Processing Order chart.

It acts as a WHERE clause for the GROUP BY and, with SQL being declarative, can be phrased slightly differently in our heads in order to easily remember the usage:

"The query HAVING grouped sales > 150,000"

Using this new keyword we can satisfy our query in its entirety:

In [ ]:
select month(orderDate) as orderMonth, salesPersonID, sum(totalDue) as totalSales
from JupyterDatabase.sales.salesOrderHeader
where orderDate between '2018-01-01' and '2018-12-31'
group by month(orderDate), salesPersonID
having sum(totalDue) > 150000
order by orderMonth, salesPersonID;